In [5]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -r requirements.txt  # install

Cloning into 'yolov5'...
remote: Enumerating objects: 10911, done.
remote: Total 10911 (delta 0), reused 0 (delta 0), pack-reused 10911
Receiving objects: 100% (10911/10911), 11.00 MiB | 12.25 MiB/s, done.
Resolving deltas: 100% (7539/7539), done.
/USER/yolov5
     |████████████████████████████████| 11.3 MB 14.2 MB/s eta 0:00:01
     |████████████████████████████████| 5.8 MB 32.2 MB/s eta 0:00:01
     |████████████████████████████████| 292 kB 35.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 32.2 MB/s eta 0:00:01
     |████████████████████████████████| 895 kB 26.1 MB/s eta 0:00:01
     |████████████████████████████████| 781 kB 25.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 29.0 MB/s eta 0:00:01
     |████████████████████████████████| 126 kB 36.3 MB/s eta 0:00:01
     |████████████████████████████████| 4.9 MB 34.1 MB/s eta 0:00:01     |██▏                             | 327 kB 34.1 MB/s eta 0:00:01
     |████████████████████████████████| 289 kB 31

In [15]:
import  pandas  as pd
label = pd.read_csv('/USER/data/augmentation.csv') 

In [16]:
label['file_name']

0                     0.png
1                     1.png
2                     2.png
3                     3.png
4                     4.png
               ...         
3220    640_flip_rotate.png
3221    641_flip_rotate.png
3222    642_flip_rotate.png
3223    643_flip_rotate.png
3224    644_flip_rotate.png
Name: file_name, Length: 3225, dtype: object

In [ ]:
label['COVID']

In [11]:
import cv2


img = cv2.imread('/USER/data/train/0.png', cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)

In [ ]:
from google.colab.patches import cv2_imshow
cv2_imshow(img)


error: OpenCV(4.5.5) :-1: error: (-5:Bad argument) in function 'imshow'
> Overload resolution failed:
>  - imshow() missing required argument 'mat' (pos 2)
>  - imshow() missing required argument 'mat' (pos 2)
>  - imshow() missing required argument 'mat' (pos 2)


In [2]:
BASE_DIR = '/USER/data/augmentation'

In [23]:
import os 
import shutil
from sklearn.model_selection import train_test_split

class Dataset:
    def __init__(self, dest_dir, base_dir=BASE_DIR, label=label, ratio=0.9):
        self.base_dir = base_dir
        self.label = label
        self.ratio = ratio
        self.dest_dir = dest_dir
    
    def split_data(self):
        self.make_label()
        file_list = [file for file in os.listdir(self.base_dir) if file.endswith('png')]
        length = len(file_list)
        # print(file_list)
        train_directory = os.path.join(self.dest_dir, "train")
        val_directory = os.path.join(self.dest_dir, "val")
        try:
            if not os.path.exists(train_directory):
                os.makedirs(train_directory)
                os.makedirs(val_directory)
                os.makedirs(os.path.join(train_directory, "images"))
                os.makedirs(os.path.join(train_directory, "labels"))
                os.makedirs(os.path.join(val_directory, "images"))
                os.makedirs(os.path.join(val_directory, "labels"))
        except OSError:
            print ('Error: Creating directory. ' +  train_directory)

        
        for index, filename in enumerate(file_list):
            if index < length * 0.9:
                shutil.copy(os.path.join(self.base_dir, filename), os.path.join(self.dest_dir, "train", "images"))
                shutil.copy(os.path.join(self.base_dir, filename[:-3]+'txt'), os.path.join(self.dest_dir, "train", "labels"))
            else:
                shutil.copy(os.path.join(self.base_dir, filename), os.path.join(self.dest_dir, "val", "images"))
                shutil.copy(os.path.join(self.base_dir, filename[:-3]+'txt'), os.path.join(self.dest_dir, "val", "labels"))


    def make_label(self):
        for index, data_row in self.label.iterrows():
            # print(data_row['file_name'])
            f = open(os.path.join(self.base_dir, data_row['file_name'][:-3]+'txt'), 'w')
            f.write(f"{data_row['COVID']} 0.5 0.5 1 1")
            f.close()
            # if index == 0:
            #     break

In [25]:
data_set = Dataset('/USER/Yolo/yolov5/data')
data_set.split_data()

train : data/train/images
val : data/val/images

nc: 2
names: ['0', '1']


In [29]:
!python /USER/Yolo/yolov5/train.py --img 640 --batch 16 --epochs 100 --data my.yaml 

wandb: Currently logged in as: donghyeon95 (use `wandb login --relogin` to force relogin)
train: weights=yolov5/yolov5s.pt, cfg=, data=my.yaml, hyp=yolov5/data/hyps/hyp.scratch.yaml, epochs=100, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: skipping check (Docker image), for updates see https://github.com/ultralytics/yolov5
YOLOv5 🚀 v6.0-244-g9cf80b7 torch 1.7.1 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=

In [31]:
!python /USER/Yolo/yolov5/detect.py --weights /USER/Yolo/yolov5/runs/train/exp2/weights/best.pt --conf 0.5 --source "/USER/data/test" --save-txt 

detect: weights=['/USER/Yolo/yolov5/runs/train/exp2/weights/best.pt'], source=/USER/data/test, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-244-g9cf80b7 torch 1.7.1 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model Summary: 213 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/100 /USER/data/test/0.png: 640x640 1 0, Done. (0.019s)
image 2/100 /USER/data/test/1.png: 640x640 1 0, Done. (0.015s)
image 3/100 /USER/data/test/10.png: 640x640 1 0, Done. (0.015s)
image 4/100 /USER/data/test/11.png: 640x640 1 0, Done. (0.015s)
image 5/100 /USER/data/test/12.png: 640x640 1 0, Done. (0.015s)
image 6/100 /USER/data/test/13.png: 

In [38]:
submission = pd.read_csv("/USER/data/sample_submission.csv")
submission

,file_name,COVID
0,0.png,0
1,1.png,0
2,2.png,0
3,3.png,0
4,4.png,0
...,...,...
95,95.png,0
96,96.png,0
97,97.png,0
98,98.png,0


In [34]:
val_dir = "/USER/Yolo/yolov5/runs/detect/exp5/labels"

In [35]:
val_list = os.listdir(val_dir)
label_list = []

for index, _ in enumerate(val_list):
    if index == 100:
        break
    f = open(os.path.join(val_dir, f"{index}.txt"), 'r')
    label_list.append(int(f.readline()[0]))
    f.close()
    
label_list

[0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1]

In [39]:

for index, num in enumerate(label_list):
    # print(num)
    submission['COVID'][index] = num

submission.to_csv('/USER/data/predictYolov5_aug.csv', index=False)

/tmp/ipykernel_79833/3881582865.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['COVID'][index] = num
